In [880]:
import numpy as np
import pickle
import math
import matplotlib.pyplot as plt
import pandas as pd
import time
import pprint
import seaborn as sns
import CMR_IA as cmr
sns.set_context('paper')
# pd.set_option('display.max_columns', None)

In [881]:
with open("../Data/simu6b_design.pkl", 'rb') as inp:
    df_study = pickle.load(inp)
    df_test = pickle.load(inp)
# df_study = df_study.loc[df_study.session < 500]
# df_test = df_test.loc[df_test.session < 500]

In [882]:
df_study

,study_itemno1,study_itemno2,study_item1,study_item2,session
0,140,838,BLUEBERRY,LOCK,0
1,1587,1240,WARDROBE,SAXOPHONE,0
2,1132,702,PROTON,HELMET,0
3,299,1282,CHILD,SHELTER,0
4,1391,1059,STRAWBERRY,PICKLE,0
...,...,...,...,...,...
11995,301,1184,CHIMNEY,REPORT,999
11996,57,965,ATTIRE,NUT,999
11997,155,1112,BOULDER,POSSUM,999
11998,106,1368,BEAR,STAKE,999


In [883]:
df_test

,test_itemno1,test_itemno2,test_item,correct_ans,order,rep,test,session,test_pos,pair_idx
0,210,-1,CABBAGE,327,1,1,1,0,1,11
1,1251,-1,SCOTCH,819,2,1,1,0,2,8
2,145,-1,BODY,1198,1,1,1,0,3,7
3,1059,-1,PICKLE,1391,2,1,1,0,4,5
4,711,-1,HOE,1519,1,1,1,0,5,9
...,...,...,...,...,...,...,...,...,...,...
23995,965,-1,NUT,57,2,1,2,999,20,11997
23996,1075,-1,PISTOL,644,2,1,2,999,21,11993
23997,1090,-1,PLUM,1609,2,1,2,999,22,11991
23998,1184,-1,REPORT,301,2,1,2,999,23,11996


### Run CMR-IA

In [884]:
s_mat = np.load('../Data/wordpools/ltp_FR_similarity_matrix.npy')

In [885]:
params = cmr.make_default_params()
params.update(
    beta_enc = 0.5,
    beta_rec = 0, 
    beta_cue = 0.45, # beta for cue
    beta_rec_new = 0.1, # beta for retrieved item
    beta_distract = 0.4,
    beta_rec_post = 0.4,
    gamma_fc = 0.35,
    gamma_cf = 0.35,
    s_fc = 0.1,
    s_cf = 0.1,
    c_thresh = 0.64,
    c_thresh_ass = 0.01,
    kappa = 0.02,
    lamb = 0.01,
    eta = 0.005,
    phi_s = 1,
    phi_d = 0.6,
    nitems_in_accumulator = 24,
    d_ass = 0.7,
    learn_while_retrieving = True,
    omega = 5,
    alpha = 1
)
params

{'beta_enc': 0.5,
 'beta_rec': 0,
 'beta_cue': 0.45,
 'beta_rec_new': 0.1,
 'beta_rec_post': 0.4,
 'beta_distract': 0.4,
 'phi_s': 1,
 'phi_d': 0.6,
 's_cf': 0.1,
 's_fc': 0.1,
 'kappa': 0.02,
 'eta': 0.005,
 'omega': 5,
 'alpha': 1,
 'c_thresh': 0.64,
 'c_thresh_ass': 0.01,
 'd_ass': 0.7,
 'lamb': 0.01,
 'rec_time_limit': 60000.0,
 'dt': 10,
 'nitems_in_accumulator': 24,
 'max_recalls': 50,
 'learn_while_retrieving': True,
 'a': 2800,
 'b': 20,
 'm': 0,
 'n': 1,
 'c1': 0,
 'No_recall': None,
 'var_enc': 1,
 'bad_enc_ratio': 1,
 'gamma_fc': 0.35,
 'gamma_cf': 0.35}

In [886]:
df_simu, f_in, f_dif = cmr.run_success_multi_sess(params, df_study, df_test, s_mat, mode='CR-CR')
df_simu['test_pos'] = np.tile(np.arange(1,25),1000)
df_simu

CMR2 Time: 27.934999704360962


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos
0,0,210,-1,327.0,1740.0,0.773733,1
1,0,1251,-1,-2.0,-2.0,0.610553,2
2,0,145,-1,1198.0,2070.0,0.677073,3
3,0,1059,-1,1391.0,2830.0,0.645806,4
4,0,711,-1,-2.0,-2.0,0.568338,5
...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1050.0,0.834651,20
23996,999,1075,-1,-2.0,-2.0,0.628033,21
23997,999,1090,-1,-2.0,-2.0,0.623327,22
23998,999,1184,-1,301.0,1000.0,0.882230,23


In [887]:
df_simu = df_simu.merge(df_test,on=['session','test_itemno1','test_itemno2', 'test_pos'])
df_simu['correct'] = df_simu.s_resp == df_simu.correct_ans
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct
0,0,210,-1,327.0,1740.0,0.773733,1,CABBAGE,327,1,1,1,11,True
1,0,1251,-1,-2.0,-2.0,0.610553,2,SCOTCH,819,2,1,1,8,False
2,0,145,-1,1198.0,2070.0,0.677073,3,BODY,1198,1,1,1,7,True
3,0,1059,-1,1391.0,2830.0,0.645806,4,PICKLE,1391,2,1,1,5,True
4,0,711,-1,-2.0,-2.0,0.568338,5,HOE,1519,1,1,1,9,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1050.0,0.834651,20,NUT,57,2,1,2,11997,True
23996,999,1075,-1,-2.0,-2.0,0.628033,21,PISTOL,644,2,1,2,11993,False
23997,999,1090,-1,-2.0,-2.0,0.623327,22,PLUM,1609,2,1,2,11991,False
23998,999,1184,-1,301.0,1000.0,0.882230,23,REPORT,301,2,1,2,11996,True


In [888]:
df_cond = df_simu.groupby(["pair_idx","test"])['order'].mean().to_frame(name='corr_rate').reset_index()
df_cond = df_cond.pivot_table(index='pair_idx',columns='test',values='corr_rate').reset_index()
df_cond.columns = ['pair_idx','test1','test2']
def cond(x):
    test1 = x['test1']
    test2 = x['test2']
    if test1 == 1 and test2 == 1:
        return 'F-F'
    elif test1 == 1 and test2 == 2:
        return 'F-B'
    elif test1 == 2 and test2 == 1:
        return 'B-F'
    elif test1 == 2 and test2 == 2:
        return 'B-B'

df_cond['cond'] = df_cond.apply(lambda x:cond(x),axis=1)
df_cond['cong'] = df_cond.apply(lambda x: 'Identical' if x['cond'] == 'F-F' or x['cond'] == 'B-B' else 'Reversed',axis=1)
df_cond

,pair_idx,test1,test2,cond,cong
0,1,2.0,1.0,B-F,Reversed
1,2,1.0,2.0,F-B,Reversed
2,3,2.0,1.0,B-F,Reversed
3,4,1.0,1.0,F-F,Identical
4,5,2.0,2.0,B-B,Identical
...,...,...,...,...,...
11995,11996,2.0,2.0,B-B,Identical
11996,11997,1.0,2.0,F-B,Reversed
11997,11998,2.0,1.0,B-F,Reversed
11998,11999,1.0,1.0,F-F,Identical


In [889]:
pairidx2cond = df_cond.loc[:,['pair_idx','cond']].set_index("pair_idx").to_dict()['cond']
pairidx2cong = df_cond.loc[:,['pair_idx','cong']].set_index("pair_idx").to_dict()['cong']
df_simu['cond'] = df_simu.apply(lambda x:pairidx2cond[x['pair_idx']],axis=1)
df_simu['cong'] = df_simu.apply(lambda x:pairidx2cong[x['pair_idx']],axis=1)
df_simu.head(24)


,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong
0,0,210,-1,327.0,1740.0,0.773733,1,CABBAGE,327,1,1,1,11,True,F-F,Identical
1,0,1251,-1,-2.0,-2.0,0.610553,2,SCOTCH,819,2,1,1,8,False,B-B,Identical
2,0,145,-1,1198.0,2070.0,0.677073,3,BODY,1198,1,1,1,7,True,F-F,Identical
3,0,1059,-1,1391.0,2830.0,0.645806,4,PICKLE,1391,2,1,1,5,True,B-B,Identical
4,0,711,-1,-2.0,-2.0,0.568338,5,HOE,1519,1,1,1,9,False,F-B,Reversed
5,0,838,-1,-2.0,-2.0,0.493001,6,LOCK,140,2,1,1,1,False,B-F,Reversed
6,0,153,-1,736.0,2200.0,0.642022,7,BOTTLE,736,1,1,1,6,True,F-B,Reversed
7,0,1587,-1,-2.0,-2.0,0.554065,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed
8,0,856,-1,607.0,2160.0,0.640356,9,MAID,607,2,1,1,10,True,B-B,Identical
9,0,702,-1,-2.0,-2.0,0.565916,10,HELMET,1132,2,1,1,3,False,B-F,Reversed


In [890]:
len(f_in[0])

24

In [891]:
sessions = np.unique(df_simu.session.to_numpy())
tmp_corr_fin = []
tmp_omean_fin = []
tmp_omax_fin = []

for sess in sessions:
    df_tmp = df_study.loc[df_study.session == sess]
    tmp1 = df_tmp.study_itemno1.to_numpy()
    tmp2 = df_tmp.study_itemno2.to_numpy()
    tmp = np.concatenate((tmp1, tmp2))
    tmp = np.sort(tmp)

    df_tmp1 = df_simu.loc[df_simu.session == sess]
    tmp_test = df_tmp1.test_itemno1
    tmp_corr = df_tmp1.correct_ans
    # tmp_test = df_simu.loc[df_simu.session == sess, 'test_itemno']
    # tmp_corr = df_simu.loc[df_simu.session == sess, 'correct_ans']
    testid = np.searchsorted(tmp, tmp_test)
    corrid = np.searchsorted(tmp, tmp_corr)

    fin_sess = f_dif[sess]
    # tmp_corr_fin = []
    # tmp_omean_fin = []
    # tmp_omax_fin = []
    for i in range(len(testid)):
        tmp_corr_fin.append(fin_sess[i][corrid[i]])
        tmp_omean_fin.append(np.mean(np.delete(fin_sess[i], corrid[i])))
        tmp_omax_fin.append(np.delete(fin_sess[i], corrid[i]).max())
    # df_simu.loc[df_simu.session == sess, 'corr_fin']= tmp_corr_fin
    # df_simu.loc[df_simu.session == sess, 'omean_fin'] = tmp_omean_fin
    # df_simu.loc[df_simu.session == sess, 'omax_fin'] = tmp_omax_fin

df_simu['corr_fin']= tmp_corr_fin
df_simu['omean_fin'] = tmp_omean_fin
df_simu['omax_fin'] = tmp_omax_fin
df_simu

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,-1,327.0,1740.0,0.773733,1,CABBAGE,327,1,1,1,11,True,F-F,Identical,-0.358441,-0.904465,-0.440273
1,0,1251,-1,-2.0,-2.0,0.610553,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,-0.490329,-1.263857,-0.500147
2,0,145,-1,1198.0,2070.0,0.677073,3,BODY,1198,1,1,1,7,True,F-F,Identical,-0.449092,-1.464459,-0.500262
3,0,1059,-1,1391.0,2830.0,0.645806,4,PICKLE,1391,2,1,1,5,True,B-B,Identical,-0.463697,-1.847756,-0.537839
4,0,711,-1,-2.0,-2.0,0.568338,5,HOE,1519,1,1,1,9,False,F-B,Reversed,-0.464014,-2.251494,-0.543316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23995,999,965,-1,57.0,1050.0,0.834651,20,NUT,57,2,1,2,11997,True,F-B,Reversed,0.007928,-2.559108,-0.259000
23996,999,1075,-1,-2.0,-2.0,0.628033,21,PISTOL,644,2,1,2,11993,False,B-B,Identical,-4.663470,-2.742071,-0.243388
23997,999,1090,-1,-2.0,-2.0,0.623327,22,PLUM,1609,2,1,2,11991,False,B-B,Identical,-0.467704,-3.169724,-0.249170
23998,999,1184,-1,301.0,1000.0,0.882230,23,REPORT,301,2,1,2,11996,True,B-B,Identical,0.061877,-3.416572,-0.450713


In [892]:
df_fin = df_simu.groupby(['cong','test'])[['corr_fin','omean_fin','omax_fin']].mean().reset_index()
df_fin['dif'] = df_fin['corr_fin'] - df_fin['omax_fin']
df_fin

,cong,test,corr_fin,omean_fin,omax_fin,dif
0,Identical,1,-0.586897,-2.486919,-0.523837,-0.063060
1,Identical,2,-0.581226,-2.413626,-0.292853,-0.288374
2,Reversed,1,-0.614589,-2.475227,-0.525182,-0.089407
3,Reversed,2,-0.597405,-2.378205,-0.296517,-0.300889


### PLOT

In [893]:
df_simu.head(24)

,session,test_itemno1,test_itemno2,s_resp,s_rt,csim,test_pos,test_item,correct_ans,order,rep,test,pair_idx,correct,cond,cong,corr_fin,omean_fin,omax_fin
0,0,210,-1,327.0,1740.0,0.773733,1,CABBAGE,327,1,1,1,11,True,F-F,Identical,-0.358441,-0.904465,-0.440273
1,0,1251,-1,-2.0,-2.0,0.610553,2,SCOTCH,819,2,1,1,8,False,B-B,Identical,-0.490329,-1.263857,-0.500147
2,0,145,-1,1198.0,2070.0,0.677073,3,BODY,1198,1,1,1,7,True,F-F,Identical,-0.449092,-1.464459,-0.500262
3,0,1059,-1,1391.0,2830.0,0.645806,4,PICKLE,1391,2,1,1,5,True,B-B,Identical,-0.463697,-1.847756,-0.537839
4,0,711,-1,-2.0,-2.0,0.568338,5,HOE,1519,1,1,1,9,False,F-B,Reversed,-0.464014,-2.251494,-0.543316
5,0,838,-1,-2.0,-2.0,0.493001,6,LOCK,140,2,1,1,1,False,B-F,Reversed,-0.531226,-2.470430,-0.530552
6,0,153,-1,736.0,2200.0,0.642022,7,BOTTLE,736,1,1,1,6,True,F-B,Reversed,-0.483515,-2.688978,-0.560963
7,0,1587,-1,-2.0,-2.0,0.554065,8,WARDROBE,1240,1,1,1,2,False,F-B,Reversed,-0.480704,-3.101214,-0.586384
8,0,856,-1,607.0,2160.0,0.640356,9,MAID,607,2,1,1,10,True,B-B,Identical,-0.479312,-3.309856,-0.528241
9,0,702,-1,-2.0,-2.0,0.565916,10,HELMET,1132,2,1,1,3,False,B-F,Reversed,-0.500751,-3.722181,-0.547194


In [894]:
dfgp_test_order = df_simu.groupby(['rep','test','order']).correct.mean().to_frame(name='corr_rate').reset_index()
dfgp_test_order

,rep,test,order,corr_rate
0,1,1,1,0.367833
1,1,1,2,0.375500
2,1,2,1,0.364500
3,1,2,2,0.375167


In [895]:
def contingency_tab(x):
    tmp_x = pd.pivot_table(x,index='pair_idx',columns='test', values= 'correct')
    test1 = pd.Categorical(tmp_x[1], categories=[1, 0])
    test2 = pd.Categorical(tmp_x[2], categories=[1, 0])
    return pd.crosstab(index=test2,columns=test1)

In [896]:
dfgp_rep_cong = df_simu.groupby(['cong']).apply(contingency_tab)
dfgp_rep_cong

col_0               1     0
cong      row_0            
Identical 1      1801   322
          0       418  3459
Reversed  1      1830   485
          0       411  3274

In [897]:
dfgp_rep_cong / 6000

col_0                   1         0
cong      row_0                    
Identical 1      0.300167  0.053667
          0      0.069667  0.576500
Reversed  1      0.305000  0.080833
          0      0.068500  0.545667

In [898]:
df_simu.groupby(['cong','test']).correct.mean()

cong       test
Identical  1       0.369833
           2       0.353833
Reversed   1       0.373500
           2       0.385833
Name: correct, dtype: float64

In [899]:
def Yule_Q(A, B, C, D):
    return (A * D - B * C) / (A * D + B * C)

In [900]:
tab_identical = dfgp_rep_cong.loc['Identical']
Yule_Q(tab_identical[1][1]+0.5,tab_identical[0][1]+0.5,tab_identical[1][0]+0.5,tab_identical[0][0]+0.5)

0.9576061460074909

In [901]:
tab_reversed = dfgp_rep_cong.loc['Reversed']
Yule_Q(tab_reversed[1][1]+0.5,tab_reversed[0][1]+0.5,tab_reversed[1][0]+0.5,tab_reversed[0][0]+0.5)

0.9354888161148757

In [902]:
Yule_Q(0.319, 0.012, 0.006, 0.663)

0.9993193709853524

In [903]:
Yule_Q(0.293, 0.122, 0.049, 0.537)

0.9267935757627709